In [5]:
import time
import numpy as np
import pandas as pd
import random
import torch as t
import copy
from fastcore.foundation import patch
from nixtla.data.ts_dataset import TimeSeriesDataset
from nixtla.data.ts_loader import TimeSeriesLoader as TimeSeriesLoaderFast
from nixtla.data.ts_loader_pinche import TimeSeriesLoader as TimeSeriesLoaderPinche
from nixtla.data.ts_loader_general import TimeSeriesLoader as TimeSeriesLoaderGeneral

from nixtla.models.nbeats import Nbeats

## READ NORD POOL ELECTRICITY PRICE FORECASTING DATASET

In [6]:
from nixtla.data.datasets.epf import EPF, EPFInfo
Y_df, X_df = EPF.load(directory='data', group=EPFInfo.groups[0])
train_outsample_mask = np.ones(len(Y_df))
train_outsample_mask[-365 * 24:] = 0
sum(train_outsample_mask)
epf_dataset = TimeSeriesDataset(Y_df=Y_df, S_df=None, X_df=X_df, ts_train_mask=train_outsample_mask)
# print('X: time series features, of shape (#series,#times,#features): \t' + str(X.shape))
# print('Y: target series (in X), of shape (#series,#times): \t \t' + str(Y.shape))
# print('S: static features, of shape (#series,#features): \t \t' + str(S.shape))
Y_df.head()

Processing dataframes ...
Creating ts tensor ...


,unique_id,ds,y
0,NP,2013-01-01 00:00:00,31.05
1,NP,2013-01-01 01:00:00,30.47
2,NP,2013-01-01 02:00:00,28.92
3,NP,2013-01-01 03:00:00,27.88
4,NP,2013-01-01 04:00:00,26.96


## TRAIN AND VALIDATION DATALOADERS

In [7]:
train_loader = TimeSeriesLoaderFast(ts_dataset=epf_dataset,
                                    model='nbeats',
                                    offset=0,
                                    window_sampling_limit=365*4*24, 
                                    input_size=7*24,
                                    output_size=24,
                                    idx_to_sample_freq=1,
                                    batch_size=1024,
                                    is_train_loader=True)

val_loader = TimeSeriesLoaderFast(ts_dataset=epf_dataset,
                                  model='nbeats',
                                  offset=0,
                                  window_sampling_limit=365*4*24, 
                                  input_size=7*24,
                                  output_size=24,
                                  idx_to_sample_freq=1,
                                  batch_size=1024,
                                  is_train_loader=False)

In [8]:
nbeatsx = Nbeats(input_size_multiplier=7,
                 output_size=24,
                 shared_weights=False,
                 activation='relu',
                 initialization='lecun_normal',
                 stack_types=['exogenous_tcn']+3*['identity'],
                 n_blocks=4*[1],
                 n_layers=4*[2],
                 n_hidden=4*[[256,256]],
                 n_harmonics=1,
                 n_polynomials=2,
                 exogenous_n_channels=9,
                 include_var_dict={'y': [-2, -3, -8],
                                   'Exogenous1': [-1, -2, -8],
                                   'Exogenous2': [-1, -2, -8],
                                   'week_day': [-1]},
                 t_cols=epf_dataset.t_cols,
                 batch_normalization=False,
                 dropout_prob_theta=0.1,
                 dropout_prob_exogenous=0.1,
                 x_s_n_hidden=0,
                 learning_rate=0.001,
                 lr_decay=0.5,
                 n_lr_decay_steps=3,
                 weight_decay=0.0000001,
                 l1_theta=0.0001,
                 n_iterations=100,
                 early_stopping=10,
                 loss='MAE',
                 frequency=24,
                 random_seed=1,
                 seasonality='H')

nbeatsx.fit(train_ts_loader=train_loader, val_ts_loader=val_loader, verbose=True, eval_steps=5)

============================== Start fitting ==============================
Number of exogenous variables: 10
Number of static variables: 0 , with dim_hidden: 0
Number of iterations: 100
Number of blocks: 4
Step: 0, Time: 1.073, Insample MAE: 4611562.00000, Outsample MAE: 21127006.00000
Step: 5, Time: 6.680, Insample MAE: 3077835.25000, Outsample MAE: 2518521.75000
Step: 10, Time: 11.890, Insample MAE: 481513.09375, Outsample MAE: 633501.00000
Step: 15, Time: 16.947, Insample MAE: 141029.14062, Outsample MAE: 76808.22656
Step: 20, Time: 22.085, Insample MAE: 10478.07520, Outsample MAE: 1846.41101
Step: 25, Time: 27.159, Insample MAE: 1857.35522, Outsample MAE: 554.22729
Step: 30, Time: 32.281, Insample MAE: 484.18457, Outsample MAE: 138.78003
Step: 35, Time: 37.308, Insample MAE: 150.51764, Outsample MAE: 13.62875
Step: 40, Time: 42.383, Insample MAE: 62.75420, Outsample MAE: 14.81495
Step: 45, Time: 47.444, Insample MAE: 25.47292, Outsample MAE: 12.88299
Step: 50, Time: 53.795, Insamp